#  <center>Project 1 - Multi-layer perceptron </center>

The objective of this project is to test different architectures to compare two digits visible in a
two-channel image. It aims at showing in particular the impact of weight sharing, and of the use of an
auxiliary loss to help the training of the main objective.
It should be implemented with PyTorch only code, in particular without using other external libraries
such as scikit-learn or numpy.


In [2]:
import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F
import dlc_practical_prologue as prologue

# Loading the data

We generate the data sets to using with the function generate_pair_sets(N) defined in the file
dlc_practical_prologue.py. This function returns six tensors:


| Name          | Tensor dimension | Type     | Content                                    |
|---------------|------------------|----------|--------------------------------------------|
| train_input   | N × 2 × 14 × 14  | float32  | Images                                     |
| train_target  | N                | int64    | Class to predict ∈ {0, 1}                  |
| train_classes | N × 2            | int64    | Classes of the two digits ∈ {0, . . . , 9} |
| test_input    | N × 2 × 14 × 14  | float32  | Images                                     |
| test_target   | N                | int64    | Class to predict ∈ {0, 1}                  |
| test_classes  | N × 2            | int64    | Classes of the two digits ∈ {0, . . . , 9} |

In [3]:
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(1000)

In [4]:

test_input.view(1000,-1).size()

torch.Size([1000, 392])

# Defining the Model

In [20]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(392, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, nb_hidden)
        self.fc3 = nn.Linear(nb_hidden, nb_hidden)
        self.fc4 = nn.Linear(nb_hidden, 2)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        x= self.sigmoid(x)
        return x


# Training the model

Fully connected Neural Network 3 hidden layers

In [21]:
batch_size = 100
n_epochs = 25
def train_model(model,train_input,train_target):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr = 0.2)
    for e in range(0,n_epochs):
        for input, targets in zip(train_input.split(batch_size),train_target.split(batch_size)):
            output = model(input)
            loss = criterion(output,targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if(e%5 ==0):
            print('epoch : ',e,' loss : ',loss.item())

In [22]:
model = Net(200)
train_model(model,train_input.view(1000,-1),train_target)

epoch :  0  loss :  0.6932509541511536
epoch :  50  loss :  0.3233296871185303
epoch :  100  loss :  0.3232756555080414
epoch :  150  loss :  0.3232681155204773
epoch :  200  loss :  0.32326576113700867


# Testing

In [26]:
def compute_nb_errors(model,data_input,data_target):
    nb_errors = 0
    for input,targets in zip(data_input.split(batch_size),data_target.split(batch_size)):
        output = model(input)
        _,predicted_classes = torch.max(output,1)
        for i in range(0,output.size(0)):
            if(predicted_classes[i]!=targets[i]):
                nb_errors = nb_errors+1
                
    return nb_errors

In [27]:
print('train_error {:.02f}% test_error {:.02f}%'.format(
    compute_nb_errors(model, train_input.view(1000,-1), train_target) / train_input.size(0) * 100,
    compute_nb_errors(model, test_input.view(1000,-1), test_target) / test_input.size(0) * 100))

train_error 1.60% test_error 19.10%


In [35]:
from torchsummary import summary
summary(model, (392))

TypeError: 'int' object is not iterable